# HDF5 for Quant

This notebook illustrates how to use HDF5 files in Python to store quant equity data.

Quant equity database require the following two features: <br> 
    1. To store backtest data, which spans over a range of dates <br>
    2. Being able to update the existing data file, by date <br> 
    
In addition, we allow input of a range of dates (trailing window): <br> 
    3. for dates, on which data is available in the data file, reload the pre-computed data <br> 
    4. for dates not in the data file, compute and update the data file <br> 
    
For production, this can easily be achive by SQL database. <br> 
For research, HDF5 file provides a quickier implementable solution for this purpose. <br> 

This notebook is organized as 
1. Fake data
2. Store to .h5 file
3. Update method, with input dates overlap with dates within data file
4. Append data on a new date 

In [60]:
import os
import collections
import datetime
import h5py

import numpy as np
import pandas as pd

In [98]:
# HELP FUNCTIONS (to .py file)

# Dates resample to month ends
def downsample_dates(dates, freq='M', col='Date'):
    df = pd.Series(dates).to_frame(name=col)
    if freq == 'M': 
        df[freq] = df[col].map(lambda d: int(100*d.year+d.month))
    else:
        raise Error('Not Yet Implemented')
    df_ends = df.groupby(freq).last()
    return df_ends[col].tolist()

def fake_fexp_data(dates, tickers, factors, info, col_date='Date', col_ticker='Ticker'):
    """Fake factor exposure data, index = (col_date, col_ticker), columns = col_factor.
    """
    dates.sort()
    dates = pd.to_datetime(dates)
    n = len(tickers) # cross-sectional length 
    
    dfs = collections.OrderedDict([(dt, None) for dt in dates])
    for date in dates: 
        f2s = collections.OrderedDict([(f, None) for f in factors])
        for f in f2s:
            bar = np.random.uniform(high=info.loc[f, 'drift'], size=n)
            vol = np.random.uniform(high=info.loc[f, 'std'], size=n)
            f2s[f] = pd.Series(bar + vol * np.random.randn(n), index=tickers)
        dfs[date] = pd.DataFrame(f2s)
    df = pd.concat(dfs)
    df.index.names = [col_date, col_ticker]
    return df

In [99]:
# Fake Data
MONTHS_PER_YEAR = 12

# dates 
start_date = pd.to_datetime('2015-01-01') 
end_date = pd.to_datetime('2022-06-30')

bt_end = pd.to_datetime('2021-12-31')

training_years = 3
freq = 'M'

# Number of stocks
# KO - Coco Cola, PEP - Pepsi, FCAU - Fiat Chrysler Auto 
tickers = ['GS', 'MS', 'C', 'BAC', 'F', 'GM', 'FCAU', 'TSLA', 'WMT', 'COST', 'KO', 'PEP']

# factors
factors = ['Value', 'Growth', 'Quality', 'Momentum', 'Stability']

df_factor = pd.DataFrame(index=factors)
df_factor['std'] = [0.001*x for x in [1, 5, 1, 10, 1]]
df_factor['drift'] = [0.001*x for x in [1, 5, 10, 1, -1]]

df_factor

,std,drift
Value,0.001,0.001
Growth,0.005,0.005
Quality,0.001,0.010
Momentum,0.010,0.001
Stability,0.001,-0.001


In [102]:
# Derived data
all_dates = pd.bdate_range(start=start_date, end=end_date, freq='B')
all_month_ends = downsample_dates(all_dates, freq=freq)

raw = fake_fexp_data(all_month_ends, tickers, factors, df_factor)
raw

Value    Growth   Quality  Momentum  Stability
Date       Ticker                                                   
2015-01-30 GS      0.000184  0.002962  0.000599  0.003579   0.000060
           MS      0.001609  0.003430  0.004437 -0.006221  -0.000536
           C       0.001681  0.003178  0.001944 -0.006634   0.000337
           BAC     0.002628  0.005253  0.009617  0.010917   0.001048
           F      -0.000105  0.000144  0.003502  0.000561   0.001221
...                     ...       ...       ...       ...        ...
2022-06-30 TSLA    0.000966  0.002807  0.003473 -0.002074  -0.000623
           WMT    -0.000361  0.005907  0.004721 -0.022563  -0.000564
           COST    0.000574  0.001322  0.001430 -0.003390   0.000325
           KO      0.001378 -0.000124  0.008626 -0.002801  -0.000179
           PEP     0.000092 -0.001647  0.003996  0.000992   0.000056

[1080 rows x 5 columns]

In [103]:
raw.reset_index()

,Date,Ticker,Value,Growth,Quality,Momentum,Stability
0,2015-01-30,GS,0.000184,0.002962,0.000599,0.003579,0.000060
1,2015-01-30,MS,0.001609,0.003430,0.004437,-0.006221,-0.000536
2,2015-01-30,C,0.001681,0.003178,0.001944,-0.006634,0.000337
3,2015-01-30,BAC,0.002628,0.005253,0.009617,0.010917,0.001048
4,2015-01-30,F,-0.000105,0.000144,0.003502,0.000561,0.001221
...,...,...,...,...,...,...,...
1075,2022-06-30,TSLA,0.000966,0.002807,0.003473,-0.002074,-0.000623
1076,2022-06-30,WMT,-0.000361,0.005907,0.004721,-0.022563,-0.000564
1077,2022-06-30,COST,0.000574,0.001322,0.001430,-0.003390,0.000325
1078,2022-06-30,KO,0.001378,-0.000124,0.008626,-0.002801,-0.000179


In [49]:
'Last Modified = {}'.format(datetime.datetime.now())

'Last Modified = 2022-07-10 00:37:52.653040'